Basada en la notebook UPF_ATP_fixed

Cálculo del costo de ir aumentando la cantidad de UPF según el modelo ME

# Protein

# <span style="color:#D291BC">Aumento y decremento de UPF</span>

## <span style="color:#957DAD">Secciones</span>

+ [Import y declaración de funciones](#Import_y_declaración_de_funciones)
+ [Resolución de modelos](#Resolución_de_modelos)


<a id="Import_y_declaración_de_funciones"></a>
## <span style="color:#957DAD">Import y declaración de funciones</span>

In [4]:
import pickle


In [7]:
# funcion para resolver el modelo en una mu específica

def solve_me_model_fixed(me, muf,using_soplex=False):
    if using_soplex:
        from cobrame.solve.algorithms import solve_at_growth_rate
        solve_at_growth_rate(me, muf)
    else:
        from qminospy.me1 import ME_NLP1
        print('qminos')
        # The object containing solveME methods--composite that uses a ME model object 
        me_nlp = ME_NLP1(me, growth_key='mu')
        # Use solv for now
        xopt,status,hs = me_nlp.solvelp(0.69, verbosity=2)
        me.solution.f = me.solution.x_dict['biomass_dilution']

<a id="Resolución_de_modelos"></a>
## <span style="color:#957DAD">Resolución de modelos</span>

In [9]:
# Valores por los que irá cambiando la UPF
valores= [0.24, .27, 0.30, .33, 0.36]

In [ ]:
# Resolver el modelo para cada uno de los valores de UPF
modelos_ox_glc = {}
for upf in valores:
    name = 'me'+str(upf).split('.')[1]
    with open('../../files/models/iJL1678b.pickle', 'rb') as f:
        model = pickle.load(f)
    # Fijar la UPF, fijar la mu y maximizar energia
    model.unmodeled_protein_fraction = upf
    model.update()
    model.prune()
    model.reactions.get_by_id('EX_o2_e').upper_bound = -18. # Fijar valores de oxígeno
    model.reactions.get_by_id('EX_o2_e').lower_bound = -18.
    model.reactions.get_by_id('EX_glc__D_e').upper_bound = -10. # Fijas valores de glc
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -10.
    model.objective = "ATPM"
    model.reactions.get_by_id("ATPM").upper_bound = 1000.   # Maximizar energía
    solve_me_model_fixed(model, 0.69)
    modelos_ox_glc[name] = model

/home/notebooks/cobrame/cobrame/core/processdata.py:256 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula


qminos


In [5]:
# # Guardar
for modelo in modelos_ox_glc:
    with open('../../files/models/ATPM_UPF_OX_GLC'+modelo+'.pickle', 'wb') as f:
        pickle.dump(modelos_ox_glc[modelo], f)